In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Telomere_Length"
cohort = "GSE16058"

# Input paths
in_trait_dir = "../../input/GEO/Telomere_Length"
in_cohort_dir = "../../input/GEO/Telomere_Length/GSE16058"

# Output paths
out_data_file = "../../output/preprocess/Telomere_Length/GSE16058.csv"
out_gene_data_file = "../../output/preprocess/Telomere_Length/gene_data/GSE16058.csv"
out_clinical_data_file = "../../output/preprocess/Telomere_Length/clinical_data/GSE16058.csv"
json_path = "../../output/preprocess/Telomere_Length/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
# 1. Let's first list the directory contents to understand what files are available
import os

print("Files in the cohort directory:")
files = os.listdir(in_cohort_dir)
print(files)

# Adapt file identification to handle different naming patterns
soft_files = [f for f in files if 'soft' in f.lower() or '.soft' in f.lower() or '_soft' in f.lower()]
matrix_files = [f for f in files if 'matrix' in f.lower() or '.matrix' in f.lower() or '_matrix' in f.lower()]

# If no files with these patterns are found, look for alternative file types
if not soft_files:
    soft_files = [f for f in files if f.endswith('.txt') or f.endswith('.gz')]
if not matrix_files:
    matrix_files = [f for f in files if f.endswith('.txt') or f.endswith('.gz')]

print("Identified SOFT files:", soft_files)
print("Identified matrix files:", matrix_files)

# Use the first files found, if any
if len(soft_files) > 0 and len(matrix_files) > 0:
    soft_file = os.path.join(in_cohort_dir, soft_files[0])
    matrix_file = os.path.join(in_cohort_dir, matrix_files[0])
    
    # 2. Read the matrix file to obtain background information and sample characteristics data
    background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
    clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
    background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
    
    # 3. Obtain the sample characteristics dictionary from the clinical dataframe
    sample_characteristics_dict = get_unique_values_by_row(clinical_data)
    
    # 4. Explicitly print out all the background information and the sample characteristics dictionary
    print("\nBackground Information:")
    print(background_info)
    print("\nSample Characteristics Dictionary:")
    print(sample_characteristics_dict)
else:
    print("No appropriate files found in the directory.")


Files in the cohort directory:
['GSE16058_family.soft.gz', 'GSE16058_series_matrix.txt.gz']
Identified SOFT files: ['GSE16058_family.soft.gz']
Identified matrix files: ['GSE16058_series_matrix.txt.gz']

Background Information:
!Series_title	"Distinctions between the stasis and telomere attrition senescence barriers in cultured human mammary epithelial cells"
!Series_summary	"Molecular distinctions between the stasis and telomere attrition senescence barriers in cultured human mammary epithelial cells"
!Series_summary	""
!Series_summary	"Normal human epithelial cells in culture have generally shown a limited proliferative potential of ~10-40 population doublings before encountering a stress-associated senescence barrier (stasis) associated with elevated levels of cyclin-dependent kinase inhibitors p16 and/or p21. We now show that simple changes in media composition can expand the proliferative potential of human mammary epithelial cells (HMEC) initiated as primary cultures to 50-60 popu

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Is gene expression data available?
is_gene_available = True  # Based on the summary and title, this dataset contains data about human mammary epithelial cells gene expression

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
trait_row = 3  # 'growth status' can be considered a proxy for telomere length
age_row = None  # No age information is available
gender_row = None  # No gender information is available

# 2.2 Data Type Conversion Functions
def convert_trait(value):
    """
    Convert telomere length-related growth status to a binary value.
    0: Short telomeres (Agonescence, Stasis, Senescent states)
    1: Normal telomeres (Growing states)
    """
    if value is None or ':' not in value:
        return None
    
    status = value.split(':', 1)[1].strip().lower()
    
    # States associated with telomere attrition or senescence (shorter telomeres)
    if 'agonesence' in status or 'stasis' in status or 'senescent' in status:
        return 0
    # States associated with normal growth (normal telomeres)
    elif 'growing' in status or 'prestasis' in status or 'postselection' in status:
        return 1
    else:
        return None

def convert_age(value):
    # Not needed as age data is not available
    return None

def convert_gender(value):
    # Not needed as gender data is not available
    return None

# 3. Save Metadata
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction (only if trait_row is not None)
if trait_row is not None:
    # Extract clinical features
    clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the dataframe
    print("Clinical Data Preview:")
    print(preview_df(clinical_df))
    
    # Save clinical data to CSV
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    clinical_df.to_csv(out_clinical_data_file, index=True)
    print(f"Clinical data saved to {out_clinical_data_file}")


Clinical Data Preview:
{'GSM402192': [0.0], 'GSM402193': [0.0], 'GSM402194': [0.0], 'GSM402195': [0.0], 'GSM402196': [0.0], 'GSM402197': [0.0], 'GSM402198': [0.0], 'GSM402199': [0.0], 'GSM402200': [0.0], 'GSM402201': [0.0], 'GSM402202': [0.0], 'GSM402203': [1.0], 'GSM402204': [0.0], 'GSM402205': [0.0], 'GSM402206': [1.0], 'GSM402207': [1.0], 'GSM402208': [1.0], 'GSM402209': [1.0], 'GSM402210': [0.0], 'GSM402211': [0.0], 'GSM402212': [0.0], 'GSM402213': [0.0], 'GSM402214': [0.0], 'GSM402215': [0.0], 'GSM402216': [0.0], 'GSM402217': [0.0], 'GSM402218': [0.0], 'GSM402219': [0.0], 'GSM402220': [0.0], 'GSM402221': [0.0], 'GSM402222': [0.0], 'GSM402223': [0.0], 'GSM402224': [0.0], 'GSM402225': [0.0], 'GSM402226': [0.0], 'GSM402227': [0.0], 'GSM402228': [1.0], 'GSM402229': [1.0], 'GSM402230': [0.0], 'GSM402231': [0.0], 'GSM402232': [1.0], 'GSM402233': [1.0], 'GSM402234': [0.0], 'GSM402235': [0.0], 'GSM402236': [1.0], 'GSM402237': [1.0], 'GSM402238': [0.0], 'GSM402239': [0.0]}
Clinical data sa

### Step 3: Gene Data Extraction

In [4]:
# Use the helper function to get the proper file paths
soft_file_path, matrix_file_path = geo_get_relevant_filepaths(in_cohort_dir)

# Extract gene expression data
try:
    gene_data = get_genetic_data(matrix_file_path)
    
    # Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
    
    # Print shape to understand the dataset dimensions
    print(f"\nGene expression data shape: {gene_data.shape}")
    
except Exception as e:
    print(f"Error extracting gene data: {e}")


First 20 gene/probe identifiers:
Index(['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at', '1294_at',
       '1316_at', '1320_at', '1405_i_at', '1431_at', '1438_at', '1487_at',
       '1494_f_at', '1598_g_at', '160020_at', '1729_at', '1773_at', '177_at',
       '179_at', '1861_at'],
      dtype='object', name='ID')

Gene expression data shape: (22277, 48)


### Step 4: Gene Identifier Review

In [5]:
# These identifiers (like '1007_s_at', '1053_at', etc.) are probe IDs from Affymetrix microarrays,
# not standard human gene symbols. They need to be mapped to official gene symbols.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
try:
    # Use the correct variable name from previous steps
    gene_annotation = get_gene_annotation(soft_file_path)
    
    # 2. Preview the gene annotation dataframe
    print("Gene annotation preview:")
    print(preview_df(gene_annotation))
    
except UnicodeDecodeError as e:
    print(f"Unicode decoding error: {e}")
    print("Trying alternative approach...")
    
    # Read the file with Latin-1 encoding which is more permissive
    import gzip
    import pandas as pd
    
    # Manually read the file line by line with error handling
    data_lines = []
    with gzip.open(soft_file_path, 'rb') as f:
        for line in f:
            # Skip lines starting with prefixes we want to filter out
            line_str = line.decode('latin-1')
            if not line_str.startswith('^') and not line_str.startswith('!') and not line_str.startswith('#'):
                data_lines.append(line_str)
    
    # Create dataframe from collected lines
    if data_lines:
        gene_data_str = '\n'.join(data_lines)
        gene_annotation = pd.read_csv(pd.io.common.StringIO(gene_data_str), sep='\t', low_memory=False)
        print("Gene annotation preview (alternative method):")
        print(preview_df(gene_annotation))
    else:
        print("No valid gene annotation data found after filtering.")
        gene_annotation = pd.DataFrame()
    
except Exception as e:
    print(f"Error extracting gene annotation data: {e}")
    gene_annotation = pd.DataFrame()


Gene annotation preview:
{'ID': ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at'], 'GB_ACC': ['U48705', 'M87338', 'X51757', 'X69699', 'L36861'], 'SPOT_ID': [nan, nan, nan, nan, nan], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['Mar 8, 2007', 'Mar 8, 2007', 'Mar 8, 2007', 'Mar 8, 2007', 'Mar 8, 2007'], 'Sequence Type': ['Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence'], 'Sequence Source': [nan, nan, nan, nan, nan], 'Target Description': ['U48705 /FEATURE=mRNA /DEFINITION=HSU48705 Human receptor tyrosine kinase DDR gene, complete cds', 'M87338 /FEATURE= /DEFINITION=HUMA1SBU Human replication factor C, 40-kDa subunit (A1) mRNA, complete cds', "X51757 /FEATURE=cds /DEFINITION=HSP70B Human heat-shock protein HSP70B' gene", 'X69699 /FEATURE= /DEFINITION=HSPAX8A H.sapiens Pax8 mRNA', 'L36861 /FEATURE=expanded_cds /DEFINITION=HUMGCAPB Homo sapiens 

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify columns containing gene identifiers and gene symbols
probe_col = "ID"  # Column in annotation that matches the gene expression data index
gene_col = "Gene Symbol"  # Column containing gene symbols

print(f"Mapping from probe column '{probe_col}' to gene symbol column '{gene_col}'")

# 2. Create gene mapping dataframe using the library function
gene_mapping = get_gene_mapping(gene_annotation, probe_col, gene_col)
print(f"Created gene mapping dataframe with shape: {gene_mapping.shape}")
print("First few rows of gene mapping:")
print(gene_mapping.head())

# 3. Apply gene mapping to convert probe-level measurements to gene-level expression
gene_data = apply_gene_mapping(gene_data, gene_mapping)
print(f"Converted gene expression data shape: {gene_data.shape}")
print("First few genes:")
print(gene_data.index[:10])

# Optional: Preview a small subset of the gene expression data
print("\nPreview of gene expression data (first 3 genes, first 3 samples):")
if not gene_data.empty:
    print(gene_data.iloc[:3, :3])


Mapping from probe column 'ID' to gene symbol column 'Gene Symbol'
Created gene mapping dataframe with shape: (21248, 2)
First few rows of gene mapping:
          ID    Gene
0  1007_s_at    DDR1
1    1053_at    RFC2
2     117_at   HSPA6
3     121_at    PAX8
4  1255_g_at  GUCA1A
Converted gene expression data shape: (13046, 48)
First few genes:
Index(['A2BP1', 'A2M', 'A4GALT', 'A4GNT', 'AAAS', 'AACS', 'AADAC', 'AAK1',
       'AAMP', 'AANAT'],
      dtype='object', name='Gene')

Preview of gene expression data (first 3 genes, first 3 samples):
        GSM402192  GSM402193  GSM402194
Gene                                   
A2BP1    6.238140   6.330799   6.347286
A2M      7.105478   5.999013   5.652930
A4GALT   5.835901   5.981849   5.896369


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the obtained gene expression data using the provided function
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Normalized gene data shape: {normalized_gene_data.shape}")
print(f"First few normalized gene symbols: {list(normalized_gene_data.index[:10])}")

# Save the normalized gene data to CSV
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Load the clinical data that was already extracted and saved in a previous step
try:
    clinical_df = pd.read_csv(out_clinical_data_file, index_col=0)
    print(f"Loaded clinical data with shape: {clinical_df.shape}")
except FileNotFoundError:
    print("Clinical data file not found. Using data from previous steps.")
    # Get the clinical data from a previous step if we can't load it
    clinical_df = clinical_data  

# 3. Link clinical and genetic data
linked_data = geo_link_clinical_genetic_data(clinical_df, normalized_gene_data)
print(f"Linked data shape: {linked_data.shape}")
print(f"Linked data column count: {len(linked_data.columns)}")
print(f"First few columns of linked data: {linked_data.columns[:10].tolist()}")

# 4. Handle missing values
linked_data = handle_missing_values(linked_data, trait)
print(f"Linked data shape after handling missing values: {linked_data.shape}")

# 5. Determine whether the trait and demographic features are biased
is_trait_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
print(f"Is trait biased: {is_trait_biased}")
print(f"Linked data shape after removing biased features: {linked_data.shape}")

# 6. Conduct quality check and save the cohort information
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=True,  
    is_trait_available=True,
    is_biased=is_trait_biased, 
    df=linked_data,
    note="Dataset contains gene expression from mammary epithelial cells at different passage levels including prestasis, intermediate, post selection and agonesence stages. Telomere length is inferred from growth status."
)

# 7. Save the linked data if it's usable
print(f"Data quality check result: {'Usable' if is_usable else 'Not usable'}")
if is_usable:
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print(f"Data not saved due to quality issues.")

Normalized gene data shape: (12700, 48)
First few normalized gene symbols: ['A2M', 'A4GALT', 'A4GNT', 'AAAS', 'AACS', 'AADAC', 'AAK1', 'AAMDC', 'AAMP', 'AANAT']


Normalized gene data saved to ../../output/preprocess/Telomere_Length/gene_data/GSE16058.csv
Loaded clinical data with shape: (1, 48)
Linked data shape: (48, 12701)
Linked data column count: 12701
First few columns of linked data: ['Telomere_Length', 'A2M', 'A4GALT', 'A4GNT', 'AAAS', 'AACS', 'AADAC', 'AAK1', 'AAMDC', 'AAMP']


Linked data shape after handling missing values: (48, 12701)
For the feature 'Telomere_Length', the least common label is '1.0' with 11 occurrences. This represents 22.92% of the dataset.
The distribution of the feature 'Telomere_Length' in this dataset is fine.

Is trait biased: False
Linked data shape after removing biased features: (48, 12701)
A new JSON file was created at: ../../output/preprocess/Telomere_Length/cohort_info.json
Data quality check result: Usable


Linked data saved to ../../output/preprocess/Telomere_Length/GSE16058.csv
